In [34]:
import requests
import pandas as pd 
import json
import numpy as np
from datetime import datetime

In [2]:
endpoint = "https://wikimedia.org/api/rest_v1"

In [3]:
def return_url(article_name):
    """ Returns url for specified name
    """
    project = "en.wikipedia.org"
    access = "all-access"
    agent = "all-agents"
    article = article_name
    granularity = "monthly"
    start = "20010101"
    end = "20170101"
    params = "/metrics/pageviews/per-article/{}/{}/{}/{}/{}/{}/{}".format(project,access,
    agent, article, granularity, start, end)
    return params

In [4]:
# headers = {'content-type': 'application/json'}
# r = requests.get(endpoint+return_params("Adolf_Hitler"), headers = headers)

In [5]:
# print(r.content.decode())
# resp_dict = json.loads(r.content.decode())

In [6]:
def get_views(name): 
    """ Returns dictionary with monthly view numbers
    """
    headers = {'content-type': 'application/json'}
    r = requests.get(endpoint+return_url(name), headers = headers)
    resp_dict = json.loads(r.content.decode())
    return resp_dict

In [7]:
def month_views(resp_dict, month, year):
    time = str(year)+str(month)+"0100"
    if 'items' not in resp_dict.keys():
#         print(resp_dict)
        return 0
    for dic in resp_dict['items']:
#         print(dic['timestamp'])
#         print("{} - {}".format(dic['timestamp'], time))
        if dic['timestamp'] == time:
            return dic['views']
    return 0 

In [8]:
def cum_views(resp_dict, year):
    """ Retruns cumulative number of views for page in specified year
    """
    if 'items' not in resp_dict.keys():
#         print(resp_dict)
        return 0
    views = 0
    for dic in resp_dict['items']:
        if dic['timestamp'][:4] == str(year):
            views += dic['views']
    return views

In [9]:
data = pd.read_json("../data/politicians_with_gender.json")
# wiki_urls = pd.read_json("../data/politicians_with_gender.json").WikiURL.values
# wiki_urls[1].split("/wiki/")[1]

In [10]:
data["url_name"] = data["WikiURL"].apply(lambda x: x.split("/wiki/")[1])

In [11]:
names_df = data[["ID", "url_name"]]
len(names_df)

69179

In [12]:
# article_names = [url.split("/wiki/")[1] for url in wiki_urls]

In [13]:
# names_df["url_name"].values

In [14]:
df1, df2, df3, df4 = np.array_split(names_df, 4)

In [15]:
# len(df4)

In [16]:
# df1.to_csv("set1.csv", index=False, encoding="utf-8")
# df2.to_csv("set2.csv", index=False, encoding="utf-8")
# df3.to_csv("set3.csv", index=False, encoding="utf-8")
# df4.to_csv("set4.csv", index=False, encoding="utf-8")

In [17]:
# path = "set1.csv"

# names = pd.read_csv(path)["url_name"].values

In [18]:
# count = 1
# lst = []
# for name in names[:100]:
#     dic = get_views(name)
# #     print(cum_views(dic, 2016))
#     sublst = [name, dic]
#     lst.append(sublst)
#     print("{} - {} - {}".format(count ,datetime.now(), name))
#     count = count + 1

In [19]:
# get_views('Barack_Obama')

In [20]:
# pd.DataFrame(lst)

In [21]:
set1 = pd.read_json("set1.json")

In [22]:
set2 = pd.read_json("set2.json")

In [23]:
set3 = pd.read_json("set3.json")

In [24]:
set4 = pd.read_json("set4.json")

In [25]:
page_views = pd.concat([set1, set2, set3, set4])

In [26]:
page_views.columns = ["url_name", "view_dict"]

In [27]:
page_views["views"] = page_views["view_dict"].apply(lambda x: cum_views(x, 2016))

In [28]:
page_views["dec2016"] = page_views['view_dict'].apply(lambda x: month_views(x, 12, 2016))

In [ ]:
# page_views = page_views.reset_index()
page_views.to_pickle("page_views")

In [30]:
result = pd.merge(data, page_views, on=['url_name', 'url_name'])

In [ ]:
# result.drop(["view_dict"],1)
# result.to_json("page_views.json")

In [37]:
result = pd.read_json("page_views.json")
# result.drop(["view_dict"],1)
# result.to_json("page_views.json")

ValueError: Expected object or value